In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.optimize import curve_fit
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

In [2]:
Data = pd.read_excel("Infiltration_example.xlsx")
Data.tail()

,date,treatment,plot,pressure,suction,time,volume
571,2021-05-03,8,4,7,-7 cm,300,64.0
572,2021-05-03,8,4,7,-7 cm,600,50.5
573,2021-05-03,8,4,7,-7 cm,900,37.5
574,2021-05-03,8,4,7,-7 cm,1200,24.5
575,2021-05-03,8,4,7,-7 cm,1500,12.0


# Note
Convert to date format to datetime and check the data types

In [3]:
Data.dtypes

date         datetime64[ns]
treatment             int64
plot                  int64
pressure              int64
suction              object
time                  int64
volume              float64
dtype: object

In [4]:
dates = [datetime(2021,3,27), datetime(2021,5,3)]

# Note
Remove rows where 'time' is 0, 300, 600, or 900

In [5]:
Data = Data[(Data['time'] != 0) & (Data['time'] != 300) & (Data['time'] != 600) & (Data['time'] != 900)]
Data

,date,treatment,plot,pressure,suction,time,volume
4,2021-03-27,2,1,1,-1 cm,1200,34.8
5,2021-03-27,2,1,1,-1 cm,1500,31.3
10,2021-03-27,2,1,5,-5 cm,1200,24.6
11,2021-03-27,2,1,5,-5 cm,1500,23.3
16,2021-03-27,2,1,7,-7 cm,1200,13.4
...,...,...,...,...,...,...,...
563,2021-05-03,8,4,1,-1 cm,1500,2.0
568,2021-05-03,8,4,5,-5 cm,1200,21.0
569,2021-05-03,8,4,5,-5 cm,1500,5.0
574,2021-05-03,8,4,7,-7 cm,1200,24.5


# Note
Calculate water flux J_w

In [6]:
# Filtering out pressures 1, 5, and 7
filtered_data = Data[Data['pressure'].isin([1, 5, 7])]

# Pivoting the data to get volumes at different times side by side for the same pressure
pivot_table = filtered_data.pivot_table(index=['date', 'treatment', 'plot', 'pressure'], 
                                        columns='time', 
                                        values='volume',
                                        aggfunc='mean').reset_index()

# Calculating the volume difference between 1200s and 1500s
pivot_table['volume_diff'] = pivot_table[1200] - pivot_table[1500] ###unit s

# Area calculation in cm^2
radius = 2.5  # radius in cm
area_cm2 = np.pi * (radius ** 2)  # area in cm^2

# Calculating J_w in cm/s, considering volume as cm^3 (mL) over the area in cm^2
pivot_table['J_w'] = pivot_table['volume_diff'] / 300 / area_cm2  # volume difference in cm^3, time in s, area in cm^2

# Display the updated table with calculated J_w values
print(pivot_table[['date', 'treatment', 'plot', 'pressure', 'J_w']])


time       date  treatment  plot  pressure       J_w
0    2021-03-27          2     1         1  0.000594
1    2021-03-27          2     1         5  0.000221
2    2021-03-27          2     1         7  0.000153
3    2021-03-27          2     2         1  0.000968
4    2021-03-27          2     2         5  0.000390
..          ...        ...   ...       ...       ...
91   2021-05-03          8     3         5  0.003141
92   2021-05-03          8     3         7  0.002462
93   2021-05-03          8     4         1  0.004414
94   2021-05-03          8     4         5  0.002716
95   2021-05-03          8     4         7  0.002122

[96 rows x 5 columns]


# Note
Calculate Ks and lambda_c

In [7]:
def infiltration_model(h, Ks, lambda_c, r):
    pi = np.pi
    return Ks * np.exp(h / lambda_c) * (1 + (4 * lambda_c) / (pi * r))

In [8]:
# Example radius in cm converted to meters
r = 2.5 / 100  # radius in meters

# Container to hold the results
results = []

# Loop through each unique combination of date, treatment, and plot
for (date, treatment, plot), group in pivot_table.groupby(['date', 'treatment', 'plot']):
    # Extract h_values and Jw_values
    h_values = group['pressure'].values  # Ensure this is in the correct units, m of H2O
    Jw_values = group['J_w'].values
    
    # Initial guesses for Ks and lambda_c
    initial_guesses = [0.01, 0.1]  # Adjust based on your data and expectations

    # Perform curve fitting
    try:
        params, cov = curve_fit(lambda h, Ks, lambda_c: infiltration_model(h, Ks, lambda_c, r),
                                h_values, Jw_values, p0=initial_guesses,
                                maxfev=10000,
                                bounds=((0, 0), (np.inf, np.inf)))  # Non-negative bounds
        results.append({
            'date': date, 
            'treatment': treatment, 
            'plot': plot, 
            'Ks': params[0], 
            'lambda_c': params[1]
        })
    except RuntimeError as e:
        print(f"Failed to fit data for plot {plot} on {date} with treatment {treatment}: {e}")

# Convert results to a DataFrame
results_df = pd.DataFrame(results)
print(results_df)

         date  treatment  plot            Ks      lambda_c
0  2021-03-27          2     1  9.144891e-08  6.326500e+01
1  2021-03-27          2     2  5.453286e-08  1.814484e+02
2  2021-03-27          2     3  1.034080e-07  6.400412e+01
3  2021-03-27          2     4  1.107804e-08  3.744679e+02
4  2021-03-27          4     1  1.551432e-07  4.275051e+01
5  2021-03-27          4     2  6.143569e-08  1.822402e+02
6  2021-03-27          4     3  4.880437e-08  1.786836e+02
7  2021-03-27          4     4  2.067852e-12  4.083652e+06
8  2021-03-27          6     1  1.381335e-07  7.514428e+01
9  2021-03-27          6     2  7.873770e-08  1.864444e+02
10 2021-03-27          6     3  2.200890e-07  7.522525e+01
11 2021-03-27          6     4  2.486843e-07  7.595127e+01
12 2021-03-27          8     1  2.175944e-07  7.512236e+01
13 2021-03-27          8     2  2.262709e-07  8.956962e+01
14 2021-03-27          8     3  3.707733e-07  8.764309e+01
15 2021-03-27          8     4  1.704195e-07  7.992278e+

# Note
Merge parameters Ks and lamda_c to oringinal dataframe

In [9]:
merged_df = pd.merge(pivot_table, results_df, on=['date', 'treatment', 'plot'])
print(merged_df)

         date  treatment  plot  pressure  1200  1500  volume_diff       J_w  \
0  2021-03-27          2     1         1  34.8  31.3          3.5  0.000594   
1  2021-03-27          2     1         5  24.6  23.3          1.3  0.000221   
2  2021-03-27          2     1         7  13.4  12.5          0.9  0.000153   
3  2021-03-27          2     2         1  39.5  33.8          5.7  0.000968   
4  2021-03-27          2     2         5  24.0  21.7          2.3  0.000390   
..        ...        ...   ...       ...   ...   ...          ...       ...   
91 2021-05-03          8     3         5  25.5   7.0         18.5  0.003141   
92 2021-05-03          8     3         7  24.0   9.5         14.5  0.002462   
93 2021-05-03          8     4         1  28.0   2.0         26.0  0.004414   
94 2021-05-03          8     4         5  21.0   5.0         16.0  0.002716   
95 2021-05-03          8     4         7  24.5  12.0         12.5  0.002122   

              Ks      lambda_c  
0   9.144891e-08  

# Note
Calculate Kh

In [10]:
# Sorting the DataFrame by 'date', 'treatment', 'pressure', and 'plot' in that order
merged_df = merged_df.sort_values(by=['date', 'treatment', 'pressure', 'plot'], ascending=[True, True, True, True])

# You mentioned sorting pressure specifically as 1, 5, 7. 
# This can be done by using a categorical data type for 'pressure' with a specific order.
merged_df['pressure'] = pd.Categorical(merged_df['pressure'], categories=[1, 5, 7], ordered=True)

# After defining the categorical type, you sort again to respect this custom order for 'pressure'
merged_df = merged_df.sort_values(by=['date', 'treatment', 'pressure', 'plot'])

# Display the sorted DataFrame
print(merged_df)


         date  treatment  plot pressure  1200  1500  volume_diff       J_w  \
0  2021-03-27          2     1        1  34.8  31.3          3.5  0.000594   
3  2021-03-27          2     2        1  39.5  33.8          5.7  0.000968   
6  2021-03-27          2     3        1  58.5  54.8          3.7  0.000628   
9  2021-03-27          2     4        1  49.0  46.0          3.0  0.000509   
1  2021-03-27          2     1        5  24.6  23.3          1.3  0.000221   
..        ...        ...   ...      ...   ...   ...          ...       ...   
94 2021-05-03          8     4        5  21.0   5.0         16.0  0.002716   
86 2021-05-03          8     1        7  22.0   5.0         17.0  0.002886   
89 2021-05-03          8     2        7  25.5   8.0         17.5  0.002971   
92 2021-05-03          8     3        7  24.0   9.5         14.5  0.002462   
95 2021-05-03          8     4        7  24.5  12.0         12.5  0.002122   

              Ks      lambda_c  
0   9.144891e-08  6.326500e+01